# Analyze slab model output 

Wenn du das Slab model laufen lassen hast musst du es danach noch mit der mixed layer depth skalieren. Ich habe dir hier auch mal die Definition von dem Inertial activity Index reinkopiert. Guck dir am besten mal das notebook NIO_mixing_paper_materials/slab_model_integrations/analysis_PM1970_CCMP_v2_inertial_activity.ipynb an hier wird der Output von den reanalyse Daten mit den PIRATA Daten  Verglichen. 

In [1]:
# parameters
slab_model_data = "../data/processed/slab/neta_grid_1x1_slab.nc"
prefclim_data = "../data/PREFCLIM/PREFCLIM_Hydrography.nc"
#location_data = "../data/external/locations.csv"

## Tech preamble

In [2]:
import cmocean
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("agg")
from matplotlib import pyplot as plt
import xarray as xr

In [127]:
# xr.set_options(display_style="text")

## Load slab-model data

In [3]:
slab_ds = xr.open_dataset(slab_model_data)

# add a time coord for 2021
#slab_ds.coords["time"] = (
#    slab_ds.time_ens.isel(year=-1, drop=True)
#    + np.timedelta64(365, "D")
#)
#
#slab_ds = slab_ds.drop(["time_ens", ])

slab_ds

<xarray.Dataset>
Dimensions:    (latitude: 41, longitude: 56, time: 8760, year: 30)
Coordinates:
  * latitude   (latitude) float32 10.0 9.0 8.0 7.0 ... -27.0 -28.0 -29.0 -30.0
  * longitude  (longitude) float32 -40.0 -39.0 -38.0 -37.0 ... 13.0 14.0 15.0
  * year       (year) int64 1990 1991 1992 1993 1994 ... 2016 2017 2018 2019
    time_ens   (time, year) datetime64[ns] ...
Dimensions without coordinates: time
Data variables:
    u_slab     (latitude, longitude, time, year) float32 ...
    v_slab     (latitude, longitude, time, year) float32 ...
    umag_slab  (latitude, longitude, time, year) float32 ...
Attributes:
    slab_model_H:  1

## Load MIMOC MLD

Load MIMOC data and add 2021 time axis.

In [5]:
prefclim_ds = xr.open_dataset(prefclim_data)

#mimoc_ds.coords["month"] = (
#    "month",
#    [np.datetime64(f"2021-{month:02d}-15") for month in range(1, 13)]
#)

display(prefclim_ds)

<xarray.Dataset>
Dimensions:  (lat: 241, lon: 201, time: 12)
Coordinates:
  * lon      (lon) float64 -30.0 -29.75 -29.5 -29.25 ... 19.25 19.5 19.75 20.0
  * lat      (lat) float64 -30.0 -29.75 -29.5 -29.25 ... 29.25 29.5 29.75 30.0
  * time     (time) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0
Data variables:
    temp     (time, lat, lon) float64 ...
    salt     (time, lat, lon) float64 ...
    mld      (time, lat, lon) float64 ...
    weight   (time, lat, lon) float64 ...

## Select MLD at slab locations / times

In [37]:
# interpolate prefclim mld on slab model grid

mld_at_slab = prefclim_ds.mld.interp(lat=slab_ds.latitude, lon=slab_ds.longitude
                                    ).drop(["lat", "lon", ])
time_month = ((slab_ds.time_ens.data.astype('datetime64[M]')-
               slab_ds.time_ens.data.astype('datetime64[Y]'))[:,0])
mld_at_slab = mld_at_slab.sel(time=time_month, method='nearest').drop('time')

display(mld_at_slab)

<xarray.DataArray 'mld' (time: 8760, latitude: 41, longitude: 56)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ..., 29.14484275,
         25.99070733, 18.22915485],
        [        nan,         nan,         nan, ..., 27.61697408,
         26.32174373, 18.18882591],
        [        nan,         nan,         nan, ..., 26.85456195,
         25.7175213 , 19.65763611]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ..., 29.14484275,
         25.99070733, 18.22915485],
        [        nan,         nan,         nan, ..., 27.61697408,
         26.32174373, 18.18882591],
        [        nan,         nan,         nan, ..., 26.85456195,
         25.7175213 , 19.65763611]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ..., 29.14484275,
         25.99070733, 18.22915485],
        [        nan,         nan,         nan, ..., 27.61697408,
         26.32174373, 18.18882591],
        [        nan,         nan,         nan, ..., 26.85456195,
         25.7175213 , 19.65763611]],

       ...,

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ..., 30.50318167,
         23.41951438, 16.3026593 ],
        [        nan,         nan,         nan, ..., 31.18107817,
         25.29854527, 18.53089579],
        [        nan,         nan,         nan, ..., 31.79175518,
         27.18866117, 20.45664483]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ..., 30.50318167,
         23.41951438, 16.3026593 ],
        [        nan,         nan,         nan, ..., 31.18107817,
         25.29854527, 18.53089579],
        [        nan,         nan,         nan, ..., 31.79175518,
         27.18866117, 20.45664483]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ..., 30.50318167,
         23.41951438, 16.3026593 ],
        [        nan,         nan,         nan, ..., 31.18107817,
         25.29854527, 18.53089579],
        [        nan,         nan,         nan, ..., 31.79175518,
         27.18866117, 20.45664483]]])
Coordinates:
  * latitude   (latitude) float32 10.0 9.0 8.0 7.0 ... -27.0 -28.0 -29.0 -30.0
  * longitude  (longitude) float32 -40.0 -39.0 -38.0 -37.0 ... 13.

## Scale slab model data with MLD

In [38]:
for dv in slab_ds.data_vars.keys():
    slab_ds[dv] /= mld_at_slab

In [39]:
slab_ds.attrs["slab_model_H"] = "scaled MLD"

## Index for inertial activity

### Definition

We want to understand the seasonality of vertical mixing at the mixed-layer base that is induced by near-inertial oscillations.  If we parametrize mixing using a Richardson-number approach, for any given stratification, the role of near-inertial oscillations can be expressed by the squared vertical shear of horizontal velocities $S^2$ at the base of the mixed layer.  In the context of the linear model outlined above, $S^2$ is determined by the squared amplitude of the slab-model velocities and scales proportional to $1/H^2$ (the suqared inverse mixed-layer depth).  To disentangle the role of the wind-variability from the role of the mixed-layer depth, we ran the slab model using a constant mixed-layer depth and then normalize the squared amplitude of the slab model by their RMS.

We coin this quantity _inertial activity_.

In [44]:
def inertial_activity(umag, ref_inact=None):
    inact = umag ** 2
    inact = inact.rename({"inertial_activity"})
    if ref_inact is not None:
        inact /= (ref_inact ** 2).mean("time") ** 0.5
    return inact

Das ist der Punkt den ich Montag erwähnt habe. Die Zeitserien werden mit dem RMS normalisiert. Hier müssen wir mal schauen wie unterschiedlich der RMS vor Angola ist. Sont müsste man vielleicht für das ganz Gebiet mit dem gleichem RMS arbeiten. Das wäre gut wenn du dir das vielleicht anschauen kannst

In [47]:
%%time
inact_angola = inertial_activity(slab_ds.umag_slab, 
                                         ref_inact=inertial_activity(slab_ds.umag_slab))

/Users/mkoerner/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


CPU times: user 7.97 s, sys: 14.8 s, total: 22.8 s
Wall time: 33.5 s
